In [18]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
from ipywidgets import interact, widgets

class BayesianClassifierVisualizer:
    def __init__(self, mean1, mean2, covariance, samples_per_class=100, seed=42):
        self.mean1 = mean1
        self.mean2 = mean2
        self.covariance = covariance
        np.random.seed(seed)
        self.samples1 = np.random.multivariate_normal(mean1, covariance, samples_per_class)
        self.samples2 = np.random.multivariate_normal(mean2, covariance, samples_per_class)

    def _calculate_boundary_params(self, mu1, mu2, sigma, prior1, prior2):
        sigma_inv = np.linalg.inv(sigma)
        w1 = sigma_inv @ mu1
        w2 = sigma_inv @ mu2
        weight_vector = w1 - w2
        bias1 = -0.5 * mu1.T @ sigma_inv @ mu1 + np.log(prior1)
        bias2 = -0.5 * mu2.T @ sigma_inv @ mu2 + np.log(prior2)
        bias_term = bias1 - bias2
        return weight_vector, bias_term

    def plot_2d(self, prior_class1=0.5):
        prior_class2 = 1 - prior_class1
        w, b = self._calculate_boundary_params(self.mean1, self.mean2, self.covariance, prior_class1, prior_class2)
        x = np.linspace(-6, 6, 200)
        y = np.linspace(-6, 6, 200)
        X, Y = np.meshgrid(x, y)
        pos = np.dstack((X, Y))
        dist1 = multivariate_normal(mean=self.mean1, cov=self.covariance)
        dist2 = multivariate_normal(mean=self.mean2, cov=self.covariance)
        pdf1 = dist1.pdf(pos)
        pdf2 = dist2.pdf(pos)

        plt.figure(figsize=(12, 8))
        plt.contour(X, Y, pdf1, levels=6, colors='blue', alpha=0.6, linewidths=1)
        plt.contour(X, Y, pdf2, levels=6, colors='red', alpha=0.6, linewidths=1)
        plt.scatter(self.samples1[:, 0], self.samples1[:, 1], c='blue', label='Class 1 samples', alpha=0.7, s=30)
        plt.scatter(self.samples2[:, 0], self.samples2[:, 1], c='red', label='Class 2 samples', alpha=0.7, s=30)
        plt.scatter(self.mean1[0], self.mean1[1], c='darkblue', s=150, marker='*', label='Class 1 mean', edgecolor='black')
        plt.scatter(self.mean2[0], self.mean2[1], c='darkred', s=150, marker='*', label='Class 2 mean', edgecolor='black')

        if w[1] != 0:
            x_vals = np.linspace(-6, 6, 100)
            y_vals = (-b - w[0] * x_vals) / w[1]
            plt.plot(x_vals, y_vals, 'k--', linewidth=2, label='Decision boundary')
        else:
            x_val = -b / w[0]
            plt.axvline(x=x_val, color='k', linestyle='--', linewidth=2, label='Decision boundary')

        plt.xlim(-6, 6)
        plt.ylim(-6, 6)
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.title(f'2D Decision Boundary (Prior Class 1: {prior_class1:.3f})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axis('equal')
        plt.show()

    def plot_3d(self, prior_class1=0.5, elevation=30, azimuth=45):
        prior_class2 = 1 - prior_class1
        sigma_inv = np.linalg.inv(self.covariance)
        x = np.linspace(-6, 6, 50)
        y = np.linspace(-6, 6, 50)
        X, Y = np.meshgrid(x, y)

        w1 = sigma_inv @ self.mean1
        b1 = -0.5 * self.mean1.T @ sigma_inv @ self.mean1 + np.log(prior_class1)
        g1 = w1[0] * X + w1[1] * Y + b1

        w2 = sigma_inv @ self.mean2
        b2 = -0.5 * self.mean2.T @ sigma_inv @ self.mean2 + np.log(prior_class2)
        g2 = w2[0] * X + w2[1] * Y + b2

        fig = plt.figure(figsize=(14, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(X, Y, g1, alpha=0.4, color='blue')
        ax.plot_surface(X, Y, g2, alpha=0.4, color='red')
        ax.scatter(self.samples1[:, 0], self.samples1[:, 1], np.zeros(len(self.samples1)), c='blue', s=40, alpha=0.8)
        ax.scatter(self.samples2[:, 0], self.samples2[:, 1], np.zeros(len(self.samples2)), c='red', s=40, alpha=0.8)
        ax.scatter(self.mean1[0], self.mean1[1], 0, c='darkblue', s=200, marker='*')
        ax.scatter(self.mean2[0], self.mean2[1], 0, c='darkred', s=200, marker='*')
        decision_fn = g1 - g2
        ax.contour(X, Y, decision_fn, levels=[0], colors='black', linewidths=3)
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.set_zlabel('Discriminant value')
        ax.set_title(f'3D Discriminant Functions (Prior Class 1: {prior_class1:.3f})')
        ax.view_init(elev=elevation, azim=azimuth)
        ax.legend()
        plt.show()

    def interactive_2d(self):
        return interact(
            self.plot_2d,
            prior_class1=widgets.FloatSlider(
                value=0.5, min=0.01, max=0.99, step=0.01, description='Prior Class 1:'
            )
        )

    def interactive_3d(self):
        return interact(
            self.plot_3d,
            prior_class1=widgets.FloatSlider(
                value=0.5, min=0.01, max=0.99, step=0.01, description='Prior Class 1:'
            )
        )

visualizer = BayesianClassifierVisualizer(
    mean1=np.array([2, 2]),
    mean2=np.array([-1, -1]),
    covariance=1.0 * np.eye(2)
)


In [19]:
visualizer.interactive_2d()

interactive(children=(FloatSlider(value=0.5, description='Prior Class 1:', max=0.99, min=0.01, step=0.01), Out…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [20]:
visualizer.interactive_3d()

interactive(children=(FloatSlider(value=0.5, description='Prior Class 1:', max=0.99, min=0.01, step=0.01), Int…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>